# Exploring Vector Database Operations in LangChain

## Install OpenAI, and LangChain dependencies

In [ ]:
!pip install langchain==0.2.0
!pip install langchain-openai==0.1.7
!pip install langchain-community==0.2.0

## Install Chroma Vector DB and LangChain wrapper

In [ ]:
!pip install langchain-chroma

## Enter Open AI API Key

In [ ]:
from getpass import getpass

OPENAI_KEY = getpass('Enter Open AI API Key: ')

## Setup Environment Variables

In [ ]:
import os

os.environ['OPENAI_API_KEY'] = OPENAI_KEY

In [ ]:
documents = ['Quantum mechanics describes the behavior of very small particles.',
 'Photosynthesis is the process by which green plants make food using sunlight.',
 "Shakespeare's plays are a testament to English literature.",
 'Artificial Intelligence aims to create machines that can think and learn.',
 'The pyramids of Egypt are historical monuments that have stood for thousands of years.']

### Open AI Embedding Models

LangChain enables us to access Open AI embedding models which include the newest models: a smaller and highly efficient `text-embedding-3-small` model, and a larger and more powerful `text-embedding-3-large` model.

In [ ]:
from langchain_openai import OpenAIEmbeddings

# details here: https://openai.com/blog/new-embedding-models-and-api-updates
openai_embed_model = OpenAIEmbeddings(model='text-embedding-3-small')

## Vector Databases

One of the most common ways to store and search over unstructured data is to embed it and store the resulting embedding vectors, and then at query time to embed the unstructured query and retrieve the embedding vectors that are 'most similar' to the embedded query. A vector database takes care of storing embedded data and performing vector search for you.

### Chroma Vector DB

[Chroma](https://docs.trychroma.com/getting-started) is a AI-native open-source vector database focused on developer productivity and happiness. Chroma is licensed under Apache 2.0.

In [ ]:
# delete vector db if exists
!rm -rf ./chroma_db

### Create a Vector DB and persist on disk

Here we initialize a connection to a Chroma vector DB client, and also we want to save to disk, so we simply initialize the Chroma client and pass the directory where we want the data to be saved to.

In [ ]:
from langchain_chroma import Chroma

# create empty vector DB
chroma_db = Chroma(collection_name='search_docs',
                   embedding_function=openai_embed_model,
                   persist_directory="./chroma_db")

We take some sample documents

In [ ]:
documents

We create document IDs to uniquely identify each document

In [ ]:
ids = ['doc_'+str(i) for i in range(len(documents))]
ids

Checking the Vector DB to see if its empty

In [ ]:
chroma_db.get()

### Adding documents to Vector DB

Here we take our texts, pass them through the Open AI embedder to get embeddings and add it to the Chroma Vector DB.

If you have documents in the LangChain `Document` format then you can use `add_documents` instead

In [ ]:
chroma_db.add_texts(texts=documents, ids=ids)

We check out Vector DB now to see these documents have been indexed successfully

In [ ]:
chroma_db.get()

Run some search queries in our Vector DB

In [ ]:
query = 'Tell me about AI'
docs = chroma_db.similarity_search_with_score(query=query, k=1)
docs

In [ ]:
query = 'Do you know about the pyramids?'
docs = chroma_db.similarity_search_with_score(query=query, k=1)
docs

In [ ]:
query = 'What is Biology?'
docs = chroma_db.similarity_search_with_score(query=query, k=1)
docs

### Adding more documents to our Vector DB

You can add new documents anytime to the vector DB as shown below

In [ ]:
new_documents = [ 'Biology is the study of living organisms and their interactions with the environment.',
 'Music therapy can aid in the mental well-being of individuals.',
 'The Milky Way is just one of billions of galaxies in the universe.',
 'Economic theories help understand the distribution of resources in society.',
 'Yoga is an ancient practice that involves physical postures and meditation.']

In [ ]:
new_ids = ['doc_'+str(i+len(ids)) for i in range(len(new_documents))]
new_ids

In [ ]:
chroma_db.add_texts(texts=new_documents, ids=new_ids)

In [ ]:
chroma_db.get()

In [ ]:
query = 'What is Biology?'
docs = chroma_db.similarity_search_with_score(query=query, k=1)
docs

### Updating documents in the Vector DB

While building toward a real application, you want to go beyond adding data, and also update and delete data.

Chroma has users provide ids to simplify the bookkeeping here and update documents as shown below using the `update_documents`function

In [ ]:
chroma_db.get(['doc_3'])

In [ ]:
from langchain_core.documents import Document

ids = ['doc_3']
texts = ['AI is known as Artificial Intelligence. Artificial Intelligence aims to create machines that can think and learn.']
documents = [Document(page_content=text, metadata={'doc': id})
                for id, text in zip(ids,texts)]
documents

In [ ]:
chroma_db.update_documents(ids=ids,documents=documents)

In [ ]:
chroma_db.get(['doc_3'])

In [ ]:
query = 'What is AI?'
docs = chroma_db.similarity_search_with_score(query=query, k=1)
docs

### Deleting documents in the Vector DB

Chroma has users provide ids to simplify the bookkeeping here and delete documents as shown below using the `delete`function

In [ ]:
chroma_db.delete(['doc_9'])

In [ ]:
chroma_db.get()

### Load Vector DB from disk

Once you have saved your DB to disk, you can load it up anytime and connect to it and run queries as shown below

In [ ]:
# load from disk
db = Chroma(persist_directory="./chroma_db",
            embedding_function=openai_embed_model,
            collection_name='search_docs')

query = 'What is AI?'
docs = db.similarity_search_with_score(query=query, k=1)
docs